<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M4_BERT_multiclass_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

dataset = pd.read_csv("https://raw.githubusercontent.com/aaubs/ds-master/main/data/HateSpeechData_pre.csv")
dataset.head()

,count,hate_speech,offensive_language,neither,class,tweet,text length,final_tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,140,woman shouldn t complain clean house amp man t...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,85,boy dats cold tyga dwn bad cuffin dat hoe st p...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,120,dawg fuck bitch start cry confuse shit
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,62,look like tranny
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,137,shit hear true faker bitch tell ya


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.7 MB/s eta 0:00:00


In [3]:
import torch
import spacy
from transformers import AutoTokenizer, AutoModel

# Load the English language model in spaCy
nlp = spacy.load("en_core_web_sm")

# Load the pre-trained BERT model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
%%time
sentence_embeddings_list = []
mean_tensor_list = []
for tweet in range(len(dataset['final_tweet'][:1000])):

  # Tokenize the input text using spaCy
  input_text = dataset['final_tweet'][tweet]
  doc = nlp(input_text)

  # Create a list to store the token embeddings
  token_embeddings = []

  # Iterate over each token in the input text
  for i, token in enumerate(doc):
    # Encode the current token as a sequence of tokens
    input_ids = torch.tensor(tokenizer.encode(token.text)).unsqueeze(0)

    # Extract the hidden state embeddings for the current token
    with torch.no_grad():
        last_hidden_states = model(input_ids)

    # Get the embeddings for the current token and Obtain the average of the hidden states for each token
    token_embedding = last_hidden_states[0].mean(dim=1)


    # Add the current token embedding to the list of token embeddings
    token_embeddings.append(token_embedding)

  # Stack the token embeddings into a tensor
  sentence_embedding = torch.stack(token_embeddings, dim=0)
  sentence_embeddings_list.append(sentence_embedding)
  mean_tensor = torch.mean(sentence_embedding, dim=0)
  mean_tensor_list.append(mean_tensor.reshape(-1).numpy())

In [12]:
sentence_embeddings_list[2].shape

torch.Size([7, 1, 768])

In [13]:
mean_tensor[0].shape

torch.Size([768])

In [14]:
import numpy as np

In [15]:
X_train = np.array(mean_tensor_list[:800])
y_train = dataset['class'][:800]
X_test = np.array(mean_tensor_list[800:1000])
y_test = dataset['class'][800:1000]

# Running the models Using BERT

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier,XGBRFClassifier

In [17]:
lr = LogisticRegression()
rf = RandomForestClassifier()
xgb = XGBClassifier()

In [18]:
%%time
lr.fit(X_train, y_train)
rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CPU times: user 12.4 s, sys: 237 ms, total: 12.6 s
Wall time: 12.2 s


XGBClassifier(objective='multi:softprob')

In [19]:
%%time
# using bert-base-uncased model
print('Model LogisticRegression' + ' ' + str(lr.score(X_test, y_test)))
print('Model RandomForestClassifier' + ' ' + str(rf.score(X_test, y_test)))
print('Model XGBClassifier' + ' ' + str(xgb.score(X_test, y_test)))

Model LogisticRegression 0.845
Model RandomForestClassifier 0.73
Model XGBClassifier 0.755
CPU times: user 33.4 ms, sys: 10 ms, total: 43.4 ms
Wall time: 33.4 ms


In [71]:
%%time
# using dmis-lab/biobert-v1.1 model
print('Model LogisticRegression' + ' ' + str(lr.score(X_test, y_test)))
print('Model RandomForestClassifier' + ' ' + str(rf.score(X_test, y_test)))
print('Model XGBClassifier' + ' ' + str(xgb.score(X_test, y_test)))

Model LogisticRegression 0.79
Model RandomForestClassifier 0.74
Model XGBClassifier 0.755
CPU times: user 39.1 ms, sys: 26.9 ms, total: 66 ms
Wall time: 41.5 ms


# Running the models Using Sentence BERT (SBERT)

In [20]:
!pip install datasets -q
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.1 MB/s eta 0:00:00


In [21]:
from datasets import load_dataset
import datasets
import pandas as pd
import scipy.spatial
import numpy as np
import os, json
import glob
import re
import torch
from sentence_transformers import SentenceTransformer, util
from tokenizers import Tokenizer
from datetime import datetime
import random
import itertools
import pickle

In [30]:
%%time
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

#Sentences are encoded by calling model.encode()
tweet_embeddings = model.encode(dataset.tweet[:1000], convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

CPU times: user 1min 5s, sys: 323 ms, total: 1min 6s
Wall time: 1min 7s


In [31]:
X_train = np.array(tweet_embeddings[:800])
y_train = dataset['class'][:800]
X_test = tweet_embeddings[800:1000]
y_test = dataset['class'][800:1000]

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier,XGBRFClassifier

In [33]:
lr = LogisticRegression()
rf = RandomForestClassifier()
xgb = XGBClassifier()

In [34]:
%%time
lr.fit(X_train, y_train)
rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)

CPU times: user 8.14 s, sys: 141 ms, total: 8.29 s
Wall time: 9.56 s


XGBClassifier(objective='multi:softprob')

In [35]:
%%time
# using all-MiniLM-L6-v2 SBERT model
print('Model LogisticRegression' + ' ' + str(lr.score(X_test, y_test)))
print('Model RandomForestClassifier' + ' ' + str(rf.score(X_test, y_test)))
print('Model XGBClassifier' + ' ' + str(xgb.score(X_test, y_test)))

Model LogisticRegression 0.91
Model RandomForestClassifier 0.83
Model XGBClassifier 0.85
CPU times: user 32.2 ms, sys: 0 ns, total: 32.2 ms
Wall time: 42.5 ms
